In [1]:
from transformers import LEDTokenizerFast

import pandas as pd
import numpy as np
from tqdm import tqdm

In [7]:
tokenizer = LEDTokenizerFast.from_pretrained("allenai/led-base-16384")
df = pd.read_csv('train-old.csv') # type: pd.DataFrame
df["box_center"] = None
df["box_length"] = None

In [8]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,box_center,box_length
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,None,None
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,None,None
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,None,None
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,None,None
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,None,None
...,...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,None,None
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,None,None
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,None,None
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,None,None


In [15]:
for i in tqdm(range(len(df))):
  id = df.iloc[i]["id"]
  discourse_start = int(df.iloc[0]["discourse_start"])
  discourse_end = int(df.iloc[0]["discourse_end"])

  with open(f'train/{id}.txt', 'r') as f:
    text = f.read()

  char_array = np.zeros( (len(text)) )
  char_array[discourse_start : discourse_end] = 1

  encoding = tokenizer(text, padding='max_length', 
                      truncation=True, return_offsets_mapping=True)
  map_token_to_char = encoding['offset_mapping']
  token_array = np.zeros(len(map_token_to_char))
  for k in range(len(token_array)):
    token_array[k] = char_array[ map_token_to_char[k][0] - 1]

  tokens = token_array.nonzero()[0]

  start = tokens[0]
  end = tokens[-1]

  center = np.mean(tokens) / len(token_array)
  length = (end - start) / len(token_array)

  print(center, length)

  df.at[i, "center"] = center
  df.at[i, "length"] = length

  0%|          | 7/144293 [00:00<37:26, 64.21it/s]

0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001739501953125 0.00286865234375
0.001556396484375 0.00274658203125
0.001556396484375 0.00274658203125
0.001556396484375 0.00274658203125
0.001556396484375 0.00274658203125
0.001373291015625 0.00238037109375
0.001373291015625 0.00238037109375


  0%|          | 25/144293 [00:00<32:01, 75.10it/s]

0.001373291015625 0.00238037109375
0.001373291015625 0.00238037109375
0.001373291015625 0.00238037109375
0.001373291015625 0.00238037109375
0.001373291015625 0.00238037109375
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625
0.00140380859375 0.00244140625


  0%|          | 42/144293 [00:00<34:15, 70.18it/s]

...

  1%|          | 1155/144293 [00:15<29:54, 79.78it/s]

0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001495361328125 0.00262451171875
0.001617431640625 0.00274658203125
0.001617431640625 0.00274658203125
0.001617431640625 0.00274658203125
0.001617431640625 0.00274658203125


  1%|          | 1161/144293 [00:15<31:39, 75.35it/s]

0.001617431640625 0.00274658203125
0.001617431640625 0.00274658203125
0.001617431640625 0.00274658203125
0.00146484375 0.0025634765625
0.00146484375 0.0025634765625


KeyboardInterrupt: 

In [14]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,box_center,box_length,center,length
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,None,None,0.00174,0.002869
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,None,None,0.00174,0.002869
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,None,None,0.00174,0.002869
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,None,None,0.00174,0.002869
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,None,None,0.00174,0.002869
...,...,...,...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,None,None,NaN,NaN
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,None,None,NaN,NaN
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,None,None,NaN,NaN
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,None,None,NaN,NaN


In [9]:
df.to_csv("train.csv")